In [1]:
pip install pulp cplex scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 11.2 MB/s eta 0:00:00


The general form of LP problems we will consider is:

        min_x  cᵀx
   subject to  Gx ≥ h
               Ax = b
               l ≤ x ≤ u

The benchmark datasets use the .mps format for LP problems so there is a function to extract the parameters from this file type into the form above. But since I haven't figured out how to access these datasets yet there is also a function to generate large feasible LP example problems in the .mps format. Finally, there is a preliminary function that implements the PDHG algorithm but not yet any of the enhancments.

In [2]:
import numpy as np
from scipy.sparse import random as sparse_random
import pulp

# This function generates large feasible LP problems to test and saves them in the .mps format
def generate_feasible_lp(num_vars=100, num_ineq=200, num_eq=50, density=0.05, mps_filename="generated_lp.mps"):

    # The number of variables in each constraint with nonzero coefficients will be roughly density * num_vars

    rng = np.random.default_rng(0)

    # Step 1: Feasible solution
    x_feas = rng.uniform(low=-10, high=10, size=(num_vars, 1))

    # Step 2: Sparse matrices (convert to dense)
    G_sparse = sparse_random(num_ineq, num_vars, density=density, format='csr', random_state=None)
    A_sparse = sparse_random(num_eq, num_vars, density=density, format='csr', random_state=None)

    G = G_sparse.toarray()
    A = A_sparse.toarray()

    # Step 3: RHS vectors
    h = G @ x_feas + rng.uniform(0.1, 5.0, size=(num_ineq, 1))
    b = A @ x_feas

    # Step 4: Bounds
    l = x_feas - rng.uniform(1, 5, size=(num_vars, 1))
    u = x_feas + rng.uniform(1, 5, size=(num_vars, 1))
    l = np.maximum(l, -1e4)
    u = np.minimum(u, 1e4)

    # Step 5: Objective
    c = rng.normal(size=(num_vars, 1))

    # Step 6: Write to MPS using pulp
    prob = pulp.LpProblem("Feasible_LP", pulp.LpMinimize)
    x_vars = [
        pulp.LpVariable(f"x{i}", lowBound=float(l[i]), upBound=float(u[i]))
        for i in range(num_vars)
    ]
    prob += pulp.lpDot(c.flatten(), x_vars)

    # Inequality constraints: Gx <= h
    for i in range(num_ineq):
        prob += pulp.lpDot(G[i], x_vars) <= float(h[i]), f"ineq_{i}"

    # Equality constraints: Ax = b
    for i in range(num_eq):
        prob += pulp.lpDot(A[i], x_vars) == float(b[i]), f"eq_{i}"

    prob.writeMPS(mps_filename)
    print(f"✅ LP written to: {mps_filename}")

In [3]:
import numpy as np
import cplex
from cplex.exceptions import CplexError

# This function extracts the parameters of the LP from the .mps format to the general form
def mps_to_standard_form(mps_file):
    try:
        cpx = cplex.Cplex(mps_file)
        cpx.set_results_stream(None)  # mute output

        # Number of variables and constraints
        num_vars = cpx.variables.get_num()
        num_constraints = cpx.linear_constraints.get_num()

        # Objective vector (c)
        c = np.array(cpx.objective.get_linear())

        # Constraint matrix
        A_full = cpx.linear_constraints.get_rows()
        senses = cpx.linear_constraints.get_senses()
        rhs = np.array(cpx.linear_constraints.get_rhs())

        A = []
        G = []
        b = []
        h = []

        for i, (row, sense, rhs_i) in enumerate(zip(A_full, senses, rhs)):
            row_vec = np.zeros(num_vars)
            for idx, val in zip(row.ind, row.val):
                row_vec[idx] = val
            if sense == 'E':  # Equality constraint
                A.append(row_vec)
                b.append(rhs_i)
            elif sense == 'G':  # Greater than or equal
                G.append(row_vec)
                h.append(rhs_i)
            elif sense == 'L':  # Less than or equal
                # convert to -Gx ≥ -h
                G.append(-row_vec)
                h.append(-rhs_i)

        A = np.array(A) if A else np.zeros((0, num_vars))
        b = np.array(b) if b else np.zeros(0)
        G = np.array(G) if G else np.zeros((0, num_vars))
        h = np.array(h) if h else np.zeros(0)

        # Bounds
        l = np.array(cpx.variables.get_lower_bounds())
        u = np.array(cpx.variables.get_upper_bounds())

        c = c.reshape(-1, 1)
        h = h.reshape(-1, 1)
        b = b.reshape(-1, 1)
        l = l.reshape(-1, 1)
        u = u.reshape(-1, 1)

        return c, G, h, A, b, l, u

    except CplexError as e:
        print("CPLEX Error:", e)
        return None

In [169]:
import numpy as np

def proj_Lam(lam):
    return lam

def pdhg(c, G, h, A, b, l, u, max_iter=1000, tol=1e-4):
    """
    Solves:
        min cᵀx s.t. Gx ≥ h, Ax = b, l ≤ x ≤ u
    using PDHG algorithm.
    """

    # Define Parameters
    K = np.vstack([G, A])
    q = np.vstack([h, b])

    eta = 0.9/np.linalg.norm(K, 2)
    omega = 10

    tau = eta/omega
    sigma = eta*omega

    m_1 = G.shape[0]
    m_2 = A.shape[0]
    n = c.shape[0]

    # Termination Parameters
    tol_prim = tol * (1 + np.linalg.norm(c))
    tol_dual = tol * (1 + np.linalg.norm(q))

    # Initialize Primal and Dual Variables
    x = np.minimum(np.maximum(np.zeros((n, 1)), l), u)
    y = np.zeros((m_1 + m_2, 1))

    for i in range(max_iter):

        # Primal update
        x_old = x.copy()
        # Project x onto box constraints l ≤ x ≤ u
        grad = c - K.T @ y
        x = np.minimum(np.maximum(x - tau * grad, l), u)

        # Dual update
        y += sigma * (q - K @ (2*x - x_old))
        y[:m_1] = np.maximum(y[:m_1], 0)  # project onto constraint y:m_1 ≥ 0

        # Check Termination Criteria Periodically
        if i%1000 == 0:

            dual_op = (q.T @ y)[0][0]
            prim_op = (c.T @ x)[0][0]

            lam_p_op = (l.T @ np.maximum(grad, 0))[0][0]
            lam_n_op = (u.T @ np.minimum(grad, 0))[0][0]

            print(dual_op + lam_p_op + lam_n_op, prim_op, i)

            #print(np.linalg.norm(np.vstack([A @ x - b, np.maximum(h - G @ x, 0)])), np.linalg.norm(grad - proj_Lam(grad)), np.abs(dual_op + lam_p_op - lam_n_op - prim_op))
            #print(tol_dual, tol_prim, tol * (1 + np.abs(dual_op + lam_p_op - lam_n_op) + np.abs(prim_op)))
            if (
                np.linalg.norm(np.vstack([A @ x - b, np.maximum(h - G @ x, 0)])) <= tol_dual
                and np.linalg.norm(grad - proj_Lam(grad)) <= tol_prim
                and np.abs(dual_op + lam_p_op + lam_n_op - prim_op) <= tol * (1 + np.abs(dual_op + lam_p_op + lam_n_op) + np.abs(prim_op))
            ):
                break

    # Returns minimal objective value, minimizer estimate in list format, and the number of iterations run
    return (c.T @ x)[0][0], x.T[0].tolist(), i + 1

In [269]:
import numpy as np

def ruiz_precondition(K, c, b, h, l, u, max_iter=50, eps=1e-6):
    m, n = K.shape
    m_ineq = h.shape[0] if h is not None else 0
    m_eq = b.shape[0] if b is not None else 0

    D1 = np.ones(m)
    D2 = np.ones(n)
    K_scaled = K.copy()

    row_mask = np.max(np.abs(K), axis=1) == 0
    col_mask = np.max(np.abs(K), axis=0) == 0

    for it in range(max_iter):
        row_norms = np.max(np.abs(K_scaled), axis=1)
        row_norms[row_norms < eps] = 1.0
        row_scale = np.sqrt(row_norms)
        D1 /= row_scale
        K_scaled = K_scaled / row_scale[:, np.newaxis]

        col_norms = np.max(np.abs(K_scaled), axis=0)
        col_norms[col_norms < eps] = 1.0
        col_scale = np.sqrt(col_norms)
        D2 /= col_scale
        K_scaled = K_scaled / col_scale[np.newaxis, :]

        row_max = np.max(np.abs(K_scaled), axis=1)
        col_max = np.max(np.abs(K_scaled), axis=0)
        if np.max(np.abs(1 - row_max)) < 0.1 and np.max(np.abs(1 - col_max)) < 0.1:
            break

    D1 = D1.reshape(-1, 1)
    D2 = D2.reshape(-1, 1)

    c_tilde = c * D2
    l_tilde = l / D2 if l is not None else None
    u_tilde = u / D2 if u is not None else None

    if h is not None:
        h_tilde = h * D1[:m_ineq]
    else:
        h_tilde = None

    if b is not None:
        b_tilde = b * D1[m_ineq:]
    else:
        b_tilde = None

    return K_scaled, c_tilde, b_tilde, h_tilde, l_tilde, u_tilde, D1, D2

In [302]:
import numpy as np

def proj_Lam(lam):
    return lam

def pdhg_precond(c, G, h, A, b, l, u, max_iter=1000, tol=1e-4):
    """
    Solves:
        min cᵀx s.t. Gx ≥ h, Ax = b, l ≤ x ≤ u
    using PDHG algorithm.
    """
    c_orig = c.copy()
    G_orig = G.copy()
    h_orig = h.copy()
    A_orig = A.copy()
    b_orig = b.copy()
    l_orig = l.copy()
    u_orig = u.copy()

    # Apply Ruiz preconditioning
    K_full = np.vstack([G, A])
    K_scaled, c, b, h, l, u, D1, D2 = ruiz_precondition(
        K_full, c, b, h, l, u
    )

    m_ineq = G.shape[0]
    G = K_scaled[:m_ineq]
    A = K_scaled[m_ineq:]

    # Define Parameters (using scaled matrices)
    K = K_scaled
    q = np.vstack([h, b])

    eta = 0.9/np.linalg.norm(K, 2)
    omega = 10

    tau = eta/omega
    sigma = eta*omega

    m_1 = G.shape[0]
    m_2 = A.shape[0]
    n = c.shape[0]

    # Termination Parameters
    tol_prim = tol * (1 + np.linalg.norm(c))
    tol_dual = tol * (1 + np.linalg.norm(q))

    # Initialize Primal and Dual Variables
    x = np.minimum(np.maximum(np.zeros((n, 1)), l), u)
    y = np.zeros((m_1 + m_2, 1))

    for i in range(max_iter):

        # Primal update
        x_old = x.copy()
        # Project x onto box constraints l ≤ x ≤ u
        grad = c - K.T @ y
        x = np.minimum(np.maximum(x - tau * grad, l), u)

        # Dual update
        y += sigma * (q - K @ (2*x - x_old))
        y[:m_1] = np.maximum(y[:m_1], 0)  # project onto constraint y:m_1 ≥ 0

        # Check Termination Criteria Periodically
        if i%1000 == 0:

            dual_op = (q.T @ y)[0][0]
            prim_op = (c.T @ x)[0][0]

            lam_p_op = (l.T @ np.maximum(grad, 0))[0][0]
            lam_n_op = (u.T @ np.minimum(grad, 0))[0][0]

            print(dual_op + lam_p_op + lam_n_op, prim_op, i)

            #print(np.linalg.norm(np.vstack([A @ x - b, np.maximum(h - G @ x, 0)])), np.linalg.norm(grad - proj_Lam(grad)), np.abs(dual_op + lam_p_op - lam_n_op - prim_op))
            #print(tol_dual, tol_prim, tol * (1 + np.abs(dual_op + lam_p_op - lam_n_op) + np.abs(prim_op)))
            if (
                np.linalg.norm(np.vstack([A @ x - b, np.maximum(h - G @ x, 0)])) <= tol_dual
                and np.linalg.norm(grad - proj_Lam(grad)) <= tol_prim
                and np.abs(dual_op + lam_p_op + lam_n_op - prim_op) <= tol * (1 + np.abs(dual_op + lam_p_op + lam_n_op) + np.abs(prim_op))
            ):
                break

    x_original = x * D2

    # Returns minimal objective value, minimizer estimate in list format, and the number of iterations run
    return (c.T @ x)[0][0], x_original.T[0].tolist(), i + 1

Testing the algorithm

In [281]:
from time import perf_counter

# Makes a timer to measure code omtimality
class Timer:
    def __enter__(self):
        self.start = perf_counter()
        return self
    def __exit__(self, *args):
        self.end = perf_counter()
        self.elapsed = self.end - self.start
        print(f"Elapsed time: {self.elapsed:.6f} seconds")


In [304]:
# Create a feasible LP problem and save to a
generate_feasible_lp(num_vars=300, num_ineq=200, num_eq=200, density=0.95, mps_filename="large_example.mps")

/tmp/ipython-input-2-2499066835.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pulp.LpVariable(f"x{i}", lowBound=float(l[i]), upBound=float(u[i]))
/tmp/ipython-input-2-2499066835.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prob += pulp.lpDot(G[i], x_vars) <= float(h[i]), f"ineq_{i}"
/tmp/ipython-input-2-2499066835.py:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prob += pulp.lpDot(A[i], x_vars) == float(b[i]), f"eq_{i}"


✅ LP written to: large_example.mps


In [305]:
import cplex

# Solve the LP using either primal simplex, dual simplex, or barrier (interior point).
# Only works for LP with no more than 1000 constraints and no more than 1000 variables
cpx = cplex.Cplex("large_example.mps")

# Times how long it takes to solve
with Timer():
    cpx.solve()

# Save the minimizer and minimal objective values for comparison
cpx_obj_val = cpx.solution.get_objective_value()
cpx_min = cpx.solution.get_values()
print("Objective value:", cpx_obj_val)
print("Minimizer: xᵀ =", cpx_min)


Selected objective sense:  MINIMIZE
Selected objective  name:  OBJ
Selected RHS        name:  RHS
Selected bound      name:  BND
Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.05 sec. (59.14 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =          -822.791241
Iteration:    62   Dual objective     =          -623.349224
Iteration:   124   Dual objective     =          -534.233867
Iteration:   203   Dual objective     =          -467.468449
Iteration:   285   Dual objective     =          -379.478005
Iteration:   389   Dual objective     =          -298.051280
Iteration:   523   Dual objective     =          -246.469556
Elapsed time: 0.315194 seconds
Objective value: -246.46955599997096
Minimizer: xᵀ = [3.0979339587493486, -4.324436280910293, 4.053208273097086, -1.1810971946910591, -3.9862199183575195, 5.317300037932471, 9.2270

In [306]:
# Extract LP parameters from generated example
with Timer():
    c, G, h, A, b, l, u = mps_to_standard_form("large_example.mps")
    pdhg_obj_val, pdhg_min, iterations = pdhg_recond(c, G, h, A, b, l, u, max_iter=1000000, tol=1e-8)
print("Objective Value:", pdhg_obj_val)
print("Iterations:", iterations)
print("Minimizer: xᵀ =",pdhg_min)

# The distance between the two minimizer solutions
# Should be small but won't be incredibly small since the vectors are high dimensional
#distance = np.linalg.norm(np.array(pdhg_min) - np.array(cpx_min))
#print("Distance:", distance)


Selected objective sense:  MINIMIZE
Selected objective  name:  OBJ
Selected RHS        name:  RHS
Selected bound      name:  BND
66803.1663831524 -46.77099809389688 0
-10321.9737077975 -106.37529866000675 1000
-4710.947740074293 -150.05593516688103 2000
-2977.3141707896775 -181.78067981950431 3000
-1832.8124799500417 -204.60933794724127 4000
-1152.3139564076982 -218.1201039724307 5000
-767.9831443411118 -227.37076057795576 6000
-619.5469512726772 -231.49674579170028 7000
-506.8883226786052 -234.1653125281397 8000
-422.4864401397822 -236.9457939167176 9000
-380.34775128367255 -238.11038492216403 10000
-360.9222331528332 -239.4113585398622 11000
-338.3268666525933 -240.77781986381078 12000
-330.2051562834349 -241.70819253401967 13000
-322.2044202927049 -242.35748205753077 14000
-314.08360474684787 -243.2616656936569 15000
-307.01886846201114 -243.75041770690768 16000
-305.3594051648113 -244.52517910540533 17000
-297.0087855604363 -244.96738871167273 18000
-301.18045820377563 -245.201294

In [307]:
# Extract LP parameters from generated example
with Timer():
    c, G, h, A, b, l, u = mps_to_standard_form("large_example.mps")
    pdhg_obj_val, pdhg_min, iterations = pdhg_precond(c, G, h, A, b, l, u, max_iter=1000000, tol=1e-8)
print("Objective Value:", pdhg_obj_val)
print("Iterations:", iterations)
print("Minimizer: xᵀ =",pdhg_min)

# The distance between the two minimizer solutions
# Should be small but won't be incredibly small since the vectors are high dimensional
#distance = np.linalg.norm(np.array(pdhg_min) - np.array(cpx_min))
#print("Distance:", distance)


Selected objective sense:  MINIMIZE
Selected objective  name:  OBJ
Selected RHS        name:  RHS
Selected bound      name:  BND
66803.1663831524 -46.77099809389688 0
-10321.9737077975 -106.37529866000675 1000
-4710.947740074293 -150.05593516688103 2000
-2977.3141707896775 -181.78067981950431 3000
-1832.8124799500417 -204.60933794724127 4000
-1152.3139564076982 -218.1201039724307 5000
-767.9831443411118 -227.37076057795576 6000
-619.5469512726772 -231.49674579170028 7000
-506.8883226786052 -234.1653125281397 8000
-422.4864401397822 -236.9457939167176 9000
-380.34775128367255 -238.11038492216403 10000
-360.9222331528332 -239.4113585398622 11000
-338.3268666525933 -240.77781986381078 12000
-330.2051562834349 -241.70819253401967 13000
-322.2044202927049 -242.35748205753077 14000
-314.08360474684787 -243.2616656936569 15000
-307.01886846201114 -243.75041770690768 16000
-305.3594051648113 -244.52517910540533 17000
-297.0087855604363 -244.96738871167273 18000
-301.18045820377563 -245.201294

In [268]:
c, G, h, A, b, l, u = mps_to_standard_form("large_example.mps")
K_full = np.vstack([G, A])



Selected objective sense:  MINIMIZE
Selected objective  name:  OBJ
Selected RHS        name:  RHS
Selected bound      name:  BND
